# Create Azure Virtual Machine
Recall that in [the previous section](03_create_azure_services.ipynb), we created an IoT Edge device identity in Azure IoT Hub. In this section, we will be creating and configuring a virtual machine (VM) to act as our IoT Edge device.

## Get Global Variables

In [1]:
import os
from env_variables import *
from sys import platform

# Get the current username, which will be used to create SSH keys for connecting to the VM
if platform == "win32": #Windows
    userName = (os.environ['USERNAME']).lower()
else: #Linux or MacOS
    userName = (os.environ['USER']).lower()

## Choose a VM Size
When creating a VM, you have the option of choosing a GPU-accelerated VM or a CPU-accelerated VM. Choose the option that works best for your future needs and requriments. For this sample, we suggest choosing either a Standard_NC6 (GPU tier) VM or a Standard_DS3_v2 (CPU tier) VM. For more information on other types of VMs or different sized machines available in Azure, visit the documentation about [virtual machine types and sizes](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general).

In the following cell, choose which type of VM you wish to create by commenting out the variable which will not be in use.

In [2]:
#vm_size="Standard_DS3_v2" #CPU tier VM
vm_size="Standard_NC6" #GPU tier VM

Verify that the VM size is available in your region. You may also view this page for a full list of [VMs by region](https://azure.microsoft.com/en-us/global-infrastructure/services/?regions=non-regional,us-east,us-east-2,us-central,us-north-central,us-south-central,us-west-central,us-west,us-west-2&products=virtual-machines). Based on your VM needs, you may need to change the `vm_size`, `--location`, and/or `resourceLocation` parameters to create your desired VM.

In [3]:
%%bash --out sizes -s "$azureSubscriptionId" "$resourceLocation"
az vm list-sizes --subscription $1 --location $2 --output json

In [4]:
import json
jsonDetails = json.loads(sizes)

for n in jsonDetails:
    if vm_size == n['name']:
        neededSpace = n['numberOfCores']
        print('\n' + n['name'])
        print("Number of Cores Required to Deploy: %s vCPUs" % neededSpace)


Standard_NC6
Number of Cores Required to Deploy: 6 vCPUs


Also verify that your subscription has enough quota to create your desired VM.

In [5]:
%%bash --out output -s "$azureSubscriptionId" "$resourceLocation"
az account set --subscription $1
az vm list-usage --location $2 --output json

In [6]:
import json
jsonDetails = json.loads(output)

needed_size = vm_size.split('_')
needed_size = needed_size[1]
needed_size = ''.join([i for i in needed_size if not i.isdigit()])

quota_remaining = 0
for n in jsonDetails:
    if (needed_size in n['localName']):
        quota_remaining = int(n['limit']) - int(n['currentValue'])
        print('\n' + n['localName'])
        print("Quota Remaining: %s vCPUs" % quota_remaining)


Standard NC Promo Family vCPUs
Quota Remaining: 36 vCPUs

Standard NC Family vCPUs
Quota Remaining: 48 vCPUs

Standard NCSv2 Family vCPUs
Quota Remaining: 0 vCPUs

Standard NCSv3 Family vCPUs
Quota Remaining: 0 vCPUs


Check if the quota remaining is greater than the size of the VM you wish to create.

If you do not enough quota, you can try any of the following:
* [Select a VM](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes) with fewer vCPUs or one in another family of VMs.
* [Delete VMs](https://docs.microsoft.com/en-us/powershell/module/azurerm.compute/remove-azurermvm?view=azurermps-6.13.0) that might not be in use to free up quota.
* [Increase quota](https://docs.microsoft.com/en-us/azure/azure-portal/supportability/per-vm-quota-requests) limits.

## Create the Virtual Machine
The code snippets below will execute a shell script to create an Ubuntu VM in your Azure subscription. 

Before starting, we will be setting the parameters to be used for creating the VM. For more details on the process of creating a VM using Azure, see the tutorial on creating a [Linux VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/quick-create-cli) or the tutorial on creating a [Windows VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-cli).

In [7]:
dns_name = iotDeviceId
public_ip_name = iotDeviceId + 'publicip'
vnet_name=iotDeviceId + 'vnet'
subnet_name=iotDeviceId + 'subnet'
vnet_prefix="192.168.0.0/16"
subnet_name="FrontEnd"
subnet_prefix="192.168.1.0/24"
nsg_name=iotDeviceId + 'nsg'
nic_name=iotDeviceId + 'nic'

# Static DNS name of the VM
vm_dns_name= iotDeviceId + "." + resourceLocation + ".cloudapp.azure.com"

In [8]:
%%bash --out output -s "$iotDeviceId" "$userName" "$vm_dns_name"

# -----------------------
# Create SSH Keys - !!! DONT RUN THIS CELL IN CASE YOU ALREADY CREATED THE VM. IT WILL RESET THE PUBLIC KEYS!!!
# -----------------------
# Create public/private ssh keys.
yes y | ssh-keygen -b 2048 -t rsa -f ~/.ssh/$1"_id_rsa" -C $2@$3 -q -N '' >/dev/null

# Remove prev. trusted host with same name (IP may be changed)
if [ -e ~/.ssh/known_hosts ]; then
ssh-keygen -f ~/.ssh/known_hosts -R $3
fi

Host lvasample5d672fiotdevid.southcentralus.cloudapp.azure.com not found in /home/visionadmin/.ssh/known_hosts


In [9]:
%%bash -s "$resourceGroupName" "$resourceLocation" "$public_ip_name" "$dns_name" "$vnet_name" "$vnet_prefix" "$subnet_name" "$subnet_prefix" "$nsg_name" "$nic_name" "$iotDeviceId" "$vm_size" "$userName"

# Create resource group
az group create                                                                             \
    --name "$1"                                                                             \
    --location "$2"

# Create a public IP address resource with a static IP address
az network public-ip create                                                                 \
   --name "$3"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --allocation-method Static                                                               \
   --dns-name "$4"                                                                                   

# Create a virtual network with one subnet
az network vnet create                                                                      \
   --name "$5"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --address-prefix "$6"                                                                    \
   --subnet-name "$7"                                                                       \
   --subnet-prefix "$8"                                                                         


az network nsg create                                                                       \
   --name "$9"                                                                              \
   --resource-group "$1"                                                                  

# Open SSH port
az network nsg rule create                                                                  \
   --resource-group "$1"                                                                    \
   --nsg-name "$9"                                                                          \
   --name "Default SSH"                                                                     \
   --destination-port-ranges 22                                                             \
   --protocol Tcp                                                                           \
   --access Allow                                                                           \
   --priority 1020

# Create a network interface connected to the VNet with a static private IP address 
# and associate the public IP address resource to the NIC.
az network nic create                                                                       \
   --name "${10}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --subnet "$7"                                                                            \
   --vnet-name "$5"                                                                         \
   --public-ip-address "$3"                                                                 \
   --network-security-group "$9"                                                                             

{
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f",
  "location": "southcentralus",
  "managedBy": null,
  "name": "lvasample5d672f",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
{
  "publicIp": {
    "ddosSettings": null,
    "dnsSettings": {
      "domainNameLabel": "lvasample5d672fiotdevid",
      "fqdn": "lvasample5d672fiotdevid.southcentralus.cloudapp.azure.com",
      "reverseFqdn": null
    },
    "etag": "W/\"51d12541-b9bf-49a3-a49d-8323c3d564e4\"",
    "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Network/publicIPAddresses/lvasample5d672fiotdevidpublicip",
    "idleTimeoutInMinutes": 4,
    "ipAddress": "13.85.76.101",
    "ipConfiguration": null,
    "ipTags": [],
    "location": "southcentralus",
    "name": "lvasample5d672fiotdevidpublicip",
    "provisioningState": "Succeeded",
    "publi

In [10]:
%%bash -s "$resourceGroupName" "$resourceLocation" "$public_ip_name" "$dns_name" "$vnet_name" "$vnet_prefix" "$subnet_name" "$subnet_prefix" "$nsg_name" "$nic_name" "$iotDeviceId" "$vm_size" "$userName"

# Create the VM
# For image param: Ubuntu 18.04-LTS image -> https://docs.microsoft.com/en-us/azure/virtual-machines/linux/cli-ps-findimage
# Instead of Password login, we are uploading Private SSH key. Update the path accrodingly if needed
az vm create                                                                                \
   --name "${11}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --storage-sku Standard_LRS                                                               \
   --os-disk-name "${11}""_osdisk"                                                          \
   --image "Canonical:UbuntuServer:18.04-LTS:latest"                                        \
   --size "${12}"                                                                           \
   --nics "${10}"                                                                           \
   --admin-username "${13}"                                                                 \
   --authentication-type ssh                                                                \
   --ssh-key-value "$(< ~/.ssh/"${11}"_id_rsa.pub)"    

{
  "fqdns": "lvasample5d672fiotdevid.southcentralus.cloudapp.azure.com",
  "id": "/subscriptions/e78c95ce-3ed7-49eb-a19e-8ebcc73bb608/resourceGroups/lvasample5d672f/providers/Microsoft.Compute/virtualMachines/lvasample5d672fiotdevid",
  "location": "southcentralus",
  "macAddress": "00-0D-3A-75-47-25",
  "powerState": "VM running",
  "privateIpAddress": "192.168.1.4",
  "publicIpAddress": "13.85.76.101",
  "resourceGroup": "lvasample5d672f",
  "zones": ""
}


In the cell above,  if you get an error such as 

```
admin user name cannot contain upper case character A-Z, special characters \/"[]:|<>+=;,?*@#()! or start with $ or -
```

then try changing the value of `$userName` to a name without the prohibited characters. Also do the same for all the cells below.

In [11]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

# Install ssh keys to remote machine
ssh -i ~/.ssh/$1"_id_rsa" -o "StrictHostKeyChecking no" $2@$3 "echo 'SSH key transferred.'"

SSH key transferred.


## Restart the VM

In [12]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF
sudo -b bash -c 'sleep 5; reboot' &>/dev/null;
EOF

Welcome to Ubuntu 18.04.4 LTS (GNU/Linux 5.3.0-1028-azure x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Mon Jul  6 21:35:05 UTC 2020

  System load:  1.1               Processes:           168
  Usage of /:   4.3% of 28.90GB   Users logged in:     0
  Memory usage: 0%                IP address for eth0: 192.168.1.4
  Swap usage:   0%


0 packages can be updated.
0 updates are security updates.




Pseudo-terminal will not be allocated because stdin is not a terminal.


## Print the SSH Connection String for the VM

In [13]:
%%bash --out sshstring -s "$iotDeviceId" "$userName" "$vm_dns_name"

# If needed, use the output command to SSH into the VM
echo ssh -i ~/.ssh/$1"_id_rsa" $2@$3

In [14]:
# save the ssh conn string into .env file
set_key(envPath, "SSH_STRING", sshstring.strip())

(True,
 'SSH_STRING',
 'ssh -i /home/visionadmin/.ssh/lvasample5d672fiotdevid_id_rsa visionadmin@lvasample5d672fiotdevid.southcentralus.cloudapp.azure.com')

> <span>[!NOTE]</span>
> If you need the SSH connection string anytime in the future to connect to your IoT Edge device VM, you can find the string in the [.env file](.env) as `SSH_STRING`. 

## Next Steps
If you experience unresolved issues, you can always [delete your resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group?tabs=azure-powershell) and re-run this section.

If all the code cells above have successfully finished running, return to the Readme page to continue.   